### **Respuestas de la Práctica Calificada 4 CC0C2**

#### **1. Atención y longitud de secuencia (4 pts)**

**(a) ¿Por qué *self-attention* maneja mejor dependencias largas que una RNN clásica?**

En una RNN "pura" (o incluso en una LSTM), la información fluye paso a paso:
token 1 -> estado 1 -> estado 2 -> ... -> estado T.

Si la dependencia es entre la palabra en la posición 5 y la de la posición 100, la señal tiene que atravesar muchas multiplicaciones de matrices. Eso genera:

* **Camino largo de gradientes** (profundidad efectiva T), lo que favorece el **desvanecimiento o explosión de gradientes**.
* Dificultad práctica para "recordar" detalles que vienen de muy atrás.

En *self-attention*, en cambio, cada token puede "atender" directamente a **todos los demás tokens de la secuencia en una sola capa**. El camino entre la posición 5 y la 100 es básicamente de **longitud 1** dentro de una capa de atención (o $\log(T)$ si piensas en varias capas apiladas). Eso:

* Facilita la propagación de gradientes.
* Permite modelar dependencias largas sin tener que "transportar" información paso a paso.
* Además, se puede paralelizar el cálculo de todas las posiciones, cosa que no pasa en RNNs.


**(b) ¿Qué se pierde sin codificación posicional? Ejemplo de dos frases.**

La atención pura sobre los embeddings de palabras, sin ninguna información de posición, es **invariante a permutaciones**: ve un *multiset* de vectores, no una secuencia ordenada.
Es decir, "perro muerde hombre" y "hombre muerde perro" se verían igual si solo se consideran las mismas tres palabras.

Sin codificación posicional, el modelo:

* No sabe **qué viene antes o después**.
* No puede distinguir cambios de significado basados en el orden.
* Empeora en tareas donde el orden importa (sintaxis, rol semántico, etc.).

Ejemplo en español:

1. "El perro muerde al hombre."
2. "El hombre muerde al perro."

Las palabras son las mismas, pero el sujeto y el objeto cambian. Sin posiciones, el modelo no puede saber quién muerde a quién.

**(c) Una idea para hacer la atención más eficiente en secuencias largas y su *trade-off*.**

Una idea clásica: **atención local/dispersa (sparse)**.

* En lugar de que cada token atienda a todos los demás (complejidad $O(n^2))$, cada token solo atiende a:

  * una **ventana local** (por ejemplo $\pm$ posiciones), y/o
  * algunos **tokens "globales"** (CLS, títulos, etc.).

Ventajas:

* Complejidad baja, por ejemplo $O(n*w)$ con $w \ll n$.
* Permite llegar a secuencias mucho más largas sin reventar memoria.

*Trade-off*:

* El modelo puede **perder dependencias muy largas y sutiles** si dos tokens distantes no están conectados por la ventana local o por los pocos tokens globales.
* En general, se gana eficiencia a costa de **aproximar** la atención completa y, por tanto, del potencial de calidad en algunos casos.


#### **2. Clasificación vs traducción con Transformers (4 pts)**

**(a) Diferencia de flujo entre Transformer para clasificación vs traducción.**

* **Clasificación de texto** :

  * Se suele usar solo la parte de **encoder** (tipo BERT).
  * Se mete la secuencia de entrada, se obtienen representaciones contextualizadas.
  * Se toma el embedding de un token especial (por ejemplo, `[CLS]`) o un *pooling* sobre todos los tokens.
  * Ese vector se pasa por una o varias capas densas para sacar una **clase** (salida de dimensión fija).

* **Traducción**:

  * Usa arquitectura **encoder-decoder**.
  * El **encoder** procesa la oración origen y produce representaciones para cada token.
  * El **decoder** genera la oración destino **paso a paso**, usando:

    * *self-attention* en los tokens generados hasta ahora (con máscara causal).
    * *cross-attention* sobre las salidas del encoder.
  * La salida es una **secuencia** de tokens, no un solo vector.


**(b) Rol de la atención encoder-decoder y del *masking* en el decoder.**

* **Atención encoder-decoder**:

  * Permite que el decoder, en cada paso, "mire" a las representaciones del encoder.
  * Funciona como un mecanismo de **alineación**: para generar la siguiente palabra en el idioma destino, el modelo pondera qué partes de la oración origen son más relevantes.
  * Esto ayuda a mantener la **fidelidad** a la entrada y a manejar reordenamientos sintácticos.

* **Masking en el decoder (máscara causal)**:

  * En el *self-attention* del decoder, se impide que la posición t vea posiciones futuras (> t).
  * Así se implementa el carácter **autoregresivo**: cada token solo depende de los anteriores.
  * Evita el "hacer trampa" viendo el futuro durante el entrenamiento.

**(c) Reutilizar el sistema de traducción para un clasificador en un idioma de bajo recurso.**

Ideas clave:

1. El encoder de un sistema de traducción ya ha aprendido una representación **semántica** razonable para el idioma de entrada (o para varios idiomas, si es multilingüe).
2. Si el sistema de traducción soporta el nuevo idioma (aunque con pocos datos), podemos reutilizar su encoder como **extractor de características**.

Un posible enfoque:

* **Compartir vocabulario** entre idiomas (BPE/SentencePiece multilingüe).
* Tomar el **encoder** del sistema de traducción (que ya ve el idioma de bajo recurso) y:

  * congelarlo total o parcialmente,
  * añadir una capa de clasificación arriba (`[CLS]` o *pooling* de los estados del encoder),
  * entrenar ese cabezal con las pocas etiquetas disponibles (few-shot).
* Para mejorar, se puede hacer algo de **preentrenamiento adicional en texto sin etiqueta** del nuevo idioma usando un objetivo tipo MLM o denoising, reutilizando el encoder.

En resumen: el encoder del traductor hace de **backbone multilingüe** y solo se añade/adapta una capa de clasificación ligera para el nuevo idioma.

#### **3. BERT, preentrenamiento y fine-tuning (5 pts)**

**(a) Objetivo de *masked language modeling* (MLM) y por qué ayuda a clasificación.**

En MLM:

* Se enmascaran aleatoriamente algunos tokens (por ejemplo, 15 %) de la secuencia.
* El modelo debe predecir estos tokens enmascarados usando el **contexto completo** (izquierda y derecha).

Esto obliga al modelo a aprender:

* Representaciones **bidireccionales** y sensibles tanto al pasado como al futuro.
* Relaciones semánticas y sintácticas profundas, porque para adivinar una palabra hay que entender qué "pega" en esa posición.

Para clasificación:

* El embedding de `[CLS]` (o de la secuencia agregada) se beneficia de ese preentrenamiento general.
* Con pocas etiquetas, basta con **ajustar una capa de clasificación** encima de esas representaciones ricas para obtener buen rendimiento.


**(b) Adaptar el modelo genérico al dominio legal con textos sin etiqueta.**

Esto es un caso típico de **preentrenamiento adicional por dominio** (*domain-adaptive pretraining* o DAPT):

1. Toma el BERT genérico preentrenado en texto general.
2. Continúa el entrenamiento usando **solo texto legal sin etiquetas**, con el mismo objetivo de MLM (enmascarar y predecir palabras).
3. Usa una tasa de aprendizaje más baja y un número moderado de pasos para no destruir el conocimiento general.

Resultado:

* El modelo aprende vocabulario, giros y patrones propios del dominio legal.
* Luego, al hacer *fine-tuning* supervisado para tareas legales (clasificación de tipo de proceso, detección de cláusulas, etc.), necesita menos datos etiquetados.

**(c) Qué cambiar "arriba" del modelo para clasificar tipo de proceso y riesgos de congelar vs actualizar.**

Para clasificar "tipo de proceso judicial":

* Se añade una **capa de clasificación** encima del embedding `[CLS]`:
  $$
  h_{\text{CLS}} \to \text{Linear} \to \text{Softmax}(N_\text{clases})
  $$
* Ese cabezal se entrena con las etiquetas disponibles.

Sobre congelar vs actualizar:

* **Congelar todo BERT y entrenar solo la capa de clasificación**:

  * Ventaja: menos riesgo de sobreajuste, muy barato.
  * Problema: si el dominio legal es muy distinto de los datos originales, la representación puede no ser suficientemente especializada.

* **Actualizar todas las capas con muy pocos datos etiquetados**:

  * Ventaja: máxima capacidad de adaptación.
  * Problema: alto riesgo de **sobreajuste** y de **olvido catastrófico** del conocimiento general; además puede volverse inestable el entrenamiento.

* **Punto intermedio habitual**:

  * Ajustar solo las últimas capas del encoder (o usar tasas de aprendizaje diferenciadas: pequeña para el encoder, mayor para el cabezal de clasificación).
  * Equilibra capacidad de adaptación y estabilidad.


**(d) "Olvido" del conocimiento general y cómo mitigarlo.**

El fenómeno es el **olvido catastrófico**:

* Al entrenar el modelo únicamente en datos legales, las actualizaciones de parámetros se alinean con ese dominio.
* El modelo puede "olvidar" patrones útiles para otros dominios o incluso perder habilidades generales (por ejemplo, entender lenguaje cotidiano).

Formas de mitigarlo:

* **Mezclar datos**: durante el preentrenamiento por dominio, usar un *mix* de texto legal + parte de texto general.
* **Regularización hacia el modelo original**:

  * L2 hacia los pesos iniciales ("no te alejes mucho del BERT base").
  * Métodos tipo **Elastic Weight Consolidation (EWC)**, que penalizan cambiar demasiado los pesos cruciales.
* **Mantener dos modelos**:

  * Uno general (sin tocar) y otro adaptado a lo legal.
  * Usar el que corresponda según la tarea.


#### **4. GPT y generación controlada (3 pts)**

**(a) Diferencia de objetivos GPT vs BERT y por qué GPT es más natural para generación.**

* **BERT**:

  * Objetivo principal: *masked language modeling* (más eventualmente NSP).
  * Es **bidireccional** y ve toda la secuencia a la vez, con algunos tokens enmascarados.
  * Excelente como **encoder** para clasificación, extracción, QA, etc., pero no está entrenado explícitamente a generar texto token por token.

* **GPT**:

  * Objetivo: **modelado de lenguaje autoregresivo**.
  * Aprende a predecir el siguiente token dado todos los anteriores:
    $$
    p(x_t \mid x_1, \dots, x_{t-1})
    $$
  * Es **unidireccional (causal)**: siempre mira hacia atrás.

Para generación:

* GPT fue entrenado precisamente en el **acto de continuar una secuencia**, que es lo que hacemos al generar texto.
* Por eso, en inferencia basta con alimentar un prompt inicial y repetir:
  *predecir el siguiente token -> añadirlo -> repetir*, lo que coincide con su objetivo de entrenamiento.


**(b) Dos mecanismos (más allá solo de "prompting") para controlar longitud y reducir alucinaciones.**

1. **Control mediante el propio decodificador (decoding constraints / hiperparámetros)**

   * Longitud:

     * Fijar `max_new_tokens`, `min_length`, o usar **penalización de longitud** (por ejemplo en *beam search*) para evitar que la probabilidad favorezca secuencias ridículamente cortas o largas.
   * Estabilidad y coherencia:

     * Ajustar **temperatura**, **top-k**, **top-p (nucleus sampling)**:

       * Temperatura baja + top-k/top-p restringidos -> menos diversidad pero más coherencia y menos "fantasía".
       * Reduce la probabilidad de saltar a regiones poco probables del espacio de salida.

2. **Decodificación condicionada/con restricciones externas (grounded decoding)**

   * Integrar el modelo con una **fuente de conocimiento**:

     * RAG: recuperar documentos relevantes y restringir la respuesta a información que aparezca en esos documentos (por ejemplo, penalizando tokens que introducen entidades no presentes en el contexto).
     * **Constrained decoding**: imponer restricciones de estilo o formato (por ejemplo, gramáticas o esquemas JSON) de modo que el modelo solo pueda producir secuencias válidas bajo unas reglas.
   * Para alucinaciones:

     * Añadir un **verificador** (otro modelo o regla) que rechace continuaciones inconsistente con los datos y fuerce replantear la respuesta.

#### **5. Adaptadores y LoRA (4 pts)**

**(a) Diferencia entre *full fine-tuning* y adaptadores/LoRA (qué se entrena).**

* **Full fine-tuning**:

  * Se actualizan prácticamente **todos los parámetros** del modelo base (todas las capas de atención, FFN, embeddings, etc.).
  * El modelo resultante es una nueva copia especializada.

* **Adaptadores**:

  * Se insertan pequeños módulos adicionales (típicamente con estructura botella: proyección a menor dimensión + no linealidad + proyección de vuelta).
  * Durante el entrenamiento, **los pesos originales del modelo se congelan** y **solo se entrenan las capas adaptadoras (y a veces LayerNorm / cabezal de salida)**.
  * En inferencia, la salida es la combinación del modelo base + adaptadores.

* **LoRA**:

  * En vez de modificar por completo cada matriz W grande, se entrena una **actualización de bajo rango**:
    $$
    W' = W + \Delta W,\quad \Delta W = A B
    $$
    donde A y B son matrices de bajo rango entrenables.
  * El W original permanece congelado; sólo **A y B** se entrenan.

En ambos casos (adaptadores, LoRA), la gran mayoría de parámetros del modelo base se mantienen fijos.


**(b) Ventajas en almacenamiento y despliegue.**

* **Almacenamiento**:

  * Full fine-tuning -> necesitas guardar una copia completa del modelo por tarea (por ejemplo, 7B parámetros por cada tarea).
  * Adaptadores/LoRA -> guardas:

    * una sola copia del modelo base,
    * más módulos pequeños por tarea (típicamente ≪ 10 % del tamaño, a veces 1-2 %).
  * Esto permite tener muchos "sabores" de modelo especializados ocupando mucho menos espacio total.

* **Despliegue**:

  * Puedes cargar el modelo base **una sola vez** en memoria y luego "montar" distintos adaptadores/LoRA según la tarea o el cliente, cambiando solo un pequeño conjunto de pesos.
  * Facilita:

    * cambiar de tarea **en caliente** (hot-swap),
    * escenarios multi-tenant,
    * actualizaciones rápidas (subir solo adaptadores, no todo el modelo).

**(c) ¿Por qué PEFT puede ser insuficiente en una tarea muy rara? Explicación técnica.**

Hay situaciones donde los cambios necesarios en el espacio de funciones son demasiado "grandes" para expresarlos con pequeñas capas adicionales o actualizaciones de bajo rango:

* **Dominio extremadamente alejado del preentrenamiento**:

  * Por ejemplo, pasar de un lenguaje natural general a un lenguaje formal simbólico muy exótico o a un dominio logístico/financiero ultra específico sin casi solapamiento de vocabulario.
  * El modelo base puede no tener las **representaciones de alto nivel necesarias**, y pequeños parches (LoRA/adapters) no son suficientes; habría que reentrenar gran parte del backbone.

* **Necesidad de cambios de alta dimensión**:

  * LoRA impone que la actualización de cada matriz sea de **bajo rango**; esto restringe el tipo de transformaciones posibles.
  * Si la adaptación requiere una transformación de los pesos que **no es bien aproximable por una matriz de bajo rango** (o por unos pocos adaptadores de tamaño pequeño), el modelo no alcanza el rendimiento deseado.

* **Cambio profundo de vocabulario o tokenización**:

  * Si la tarea rara exige un vocabulario muy distinto (por ejemplo, nuevo alfabeto, símbolos técnicos, lenguaje multimodal), quizá haya que cambiar embeddings, capas de entrada/salida u otras partes estructurales que PEFT normalmente no toca.

En resumen, PEFT es excelente cuando la tarea es "cercana" al conocimiento ya codificado en el modelo. Cuando la tarea implica un **salto grande en distribución o en capacidades**, puede ser necesario un *fine-tuning* más intrusivo (o incluso reentrenamiento parcial/mayor) para introducir esas habilidades nuevas.
